In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

/home/armughan/anaconda3/envs/py3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
class CNN_Model():
    def __init__(self,params):
        #STORING PARAMETER VALUES 
        self.input_shape=params['input_shape']
        self.num_outputs=params['num_outputs']
        self.layer_hierarchy=params['layer_hierarchy']
        self.activation_fn=params.get('activation_fn',tf.nn.relu)
        self.loss_fn=params.get('loss_fn',tf.losses.softmax_cross_entropy)
        self.learning_rate=params['learning_rate']
        self.optimizer_fn=params['optimizer_fn']
        self.initializer_fn=params['initializer_fn']
        self.name_scope=params['name_scope']
        logdir=params['logdir']

    def form_placeholder(self,shape,dt=tf.float32):
        X=tf.placeholder(dt,shape=shape)
        return X
    def form_convolutional_layer(self,inputs,layer_params):
        return tf.layers.conv2d(
                    inputs=inputs,
                    filters=layer_params['num_filters'],
                    kernel_size=layer_params['kernel_size'],
                    strides=layer_params['kernel_strides'],
                    padding=layer_params['padding'],
                    kernel_initializer=self.initializer_fn(),
                    activation=None)
    def form_max_pooling_layer(self,inputs,layer_params):
        tf.layers.max_pooling2d(
                    inputs=inputs,
                    pool_size=layer_params['pool_size'],
                    strides=layer_params['pool_strides'])
    def form_activation_layer(self,inputs):
        return self.activation_fn(inputs)
    
    def form_fc_layer(self,inputs,layer_params):
        return tf.layers.dense(inputs,layer_params['num_hidden_units'],activation=None,kernel_initializer=self.initializer_fn())
    
    def form_loss(self,logits,targets):
        entropies=self.loss_fn(onehot_labels=targets,logits=logits)
        return entropies
    
    def build_model(self):
        with tf.name_scope(self.name_scope):
            self.X=self.form_placeholder(self.input_shape)
            self.Y=self.form_placeholder((None,self.num_outputs),tf.int32)
            inputs=self.X
            for layer_params in self.layer_hierarchy:
                if layer_params['layer_type']=='conv_layer':
                    inputs=self.form_convolutional_layer(inputs,layer_params)
                elif layer_params['layer_type']=='fc_layer':
                    inputs=self.form_fc_layer(inputs,layer_params)
                elif layer_params['layer_type']=='activation_layer':
                    inputs=self.form_activation_layer(inputs)
                elif layer_params['layer_type']=='pooling_layer':
                    inputs=self.form_max_pooling_layer(inputs,layer_params)
                elif layer_params['layer_type']=='flattening_layer':
                    inputs=tf.contrib.layers.flatten(inputs)

    #         making logits layer (final output layer)
            self.logits=tf.layers.dense(inputs,self.num_outputs,activation=None,kernel_initializer=self.initializer_fn())

            self.loss=self.form_loss(self.logits,self.Y)
            optimizer=self.optimizer_fn(learning_rate=self.learning_rate)
            self.train_op=optimizer.minimize(loss = self.loss)
            self.saver=tf.train.Saver(max_to_keep=4)
            self.initializer=tf.global_variables_initializer()
    def save_model(sess,step):
        if self.saved_before:
            self.saver.save(sess,self.name_scope, global_step=step,write_meta_graph=False)
        else:
            self.saver.save(sess,self.name_scope, global_step=step)#writes meta graph too
            self.saved_before=True
#         self.saver

In [3]:
params={
    'input_shape':[None, 190, 35, 1],
    'num_outputs':3,
    
    'layer_hierarchy':[
        {'layer_type':'conv_layer','kernel_size':3,'kernel_strides':1,'num_filters':5,'padding':'valid'},
        {'layer_type':'activation_layer'},
        {'layer_type':'conv_layer','kernel_size':3,'kernel_strides':1,'num_filters':5,'padding':'valid'},
        {'layer_type':'activation_layer'},
        {'layer_type':'flattening_layer'},
        {'layer_type':'fc_layer','num_hidden_units':100},
        {'layer_type':'activation_layer'},
        {'layer_type':'fc_layer','num_hidden_units':100},
        {'layer_type':'activation_layer'},
        {'name_scope':'neural_network_v1'}
    ],
    'initializer_fn':tf.contrib.layers.variance_scaling_initializer,
    'activation_fn':tf.nn.relu,
    'loss_fn':tf.losses.softmax_cross_entropy,
    'learning_rate':0.001,
    'optimizer_fn':tf.train.AdamOptimizer,
    'logdir':'/tf_logs_rnn/run/'
}

In [4]:
#testing model
tf.reset_default_graph()
model=CNN_Model(params)
model.build_model()

In [23]:
n_iter = 20
batch_size = 120
fileName = 'training_data.npy'
training_data = list(np.load(fileName))


totalFrames = len(training_data)

print (training_data[][0])

# with tf.Session() as sess:
#     model=CNN_Model(params)
#     for iteration in range(n_iter):
#         print ('current iteration '+str(iteration))
#         feed_dic = {train}

SyntaxError: invalid syntax (<ipython-input-23-3ebe8de82f60>, line 9)